In [1]:
import torch
import torch.nn as nn
import os
import pandas as pd
import pickle
from torch.utils.data import Dataset, DataLoader
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import torch.nn.functional as F

In [2]:
class EmptyArgs: 
    pass

args = EmptyArgs()

# General arguments
args.data_folder='dataset/'
args.log_dir='log'
args.seed=21
args.epochs=1
args.gpu_num=0

# Model specific arguments
args.model_type='GTM'
args.use_trends=1
args.use_img=1
args.use_text=1
args.trend_len=52
args.num_trends=3
args.batch_size=128
args.embedding_dim=32
args.hidden_dim=64
args.output_dim=12
args.use_encoder_mask=1
args.autoregressive=0
args.num_attn_heads=4
args.num_hidden_layers=1

# wandb arguments
args.wandb_entity='bonbak'
args.wandb_proj='sflab-gtm'
args.wandb_run='Run1'

args.sales_total_len=12
args.only_4weeks_loss='store_true'

args.use_encoder_mask = False
args.trend_len = 64  # 52
args.prepo_data_folder = "/home/smart01/SFLAB/sanguk/mind_br_data_prepro/"
args.data_folder = "/home/smart01/SFLAB/sanguk/mind_br_data/"
args.text_embedder = 'klue/bert-base'
args.sales_total_len = 12 # 52  # 12
args.seq_len = args.sales_total_len
args.output_dim = args.sales_total_len
args.autoregressive = 1
args.scaler = "standard" # "Minmax"
args.learning_rate = 0.0001
args.lead_time = 2
args.no_scaling = False
args.ahead_step = 6
args.val_output_week = 12
args.val_output_month = 3
args.num_attn_heads = 8
args.hidden_dim = 512
args.embedding_dim = 256
args.only_4weeks_loss = False # True
args.num_hidden_layers = 2

# args.model_type = "ANTM_5epoch---231101-0845_nonautoregressive"
args.model_type = "encoder_boxcox_train_plain_loss_test11_afterencoder_meta_200epoch_0103dataset_optimallmda_centering_2z"
args.autoregressive_train = False
args.teacher_forcing = True
args.epochs = 1000 # 500  # 300  # 150  # 500 # 50 # 300  # 5  # 500  # 100 # 5  # 100
args.batch_size = 16  # 64
args.gpu_num = 2
args.mean_scalers = True
args.mean_sensi = 1 # 2
args.std_sensi = 1 # 1.2
args.mean_sensi_inference =  1  # 2
args.std_sensi_inference =  1  # 1.2
args.before_meta = False # True #

# 1.2. 기준 optimal lamda
# args.mu_lmda = 0.0616
# args.std_lmda = 0.1258
# args.mu_centering = 0  # 0.75
# args.std_centering = 0  # 0.6

# 1.2.기준 my lamda
# args.mu_lmda =  0.19
# args.std_lmda =  0.24
# args.mu_centering =  0.5
# args.std_centering = 0.4

# 1.3. 기준(train에만 일방적으로 추가)
args.mu_lmda = 0.0619
args.std_lmda = 0.1241
args.mu_centering = 0.75
args.std_centering = 0.6

In [3]:
data_path = '/home/smart01/SFLAB/su_GTM_t/GTM_T_sanguk/'

meta_df = pd.read_csv(os.path.join(data_path,'240109_all_meta_sales_total.csv'))

with open(os.path.join(data_path, '12salesweek_test_item_number296.pkl'), 'rb') as f:
    test_ids = pickle.load(f)
test_ids = test_ids.drop('MTPT6102')

train_df = meta_df[~meta_df['item_number'].isin(test_ids)] 
test_df = meta_df[meta_df['item_number'].isin(test_ids)]

In [4]:
class MetaDataset(Dataset):
    def __init__(self, df, opt_lambda=None):
        self.df = df.reset_index(drop=True)
        self.fab = torch.Tensor(self.df.loc[:,self.df.columns.str.startswith('fabric')].values)
        self.cat = torch.Tensor(self.df.loc[:,self.df.columns.str.startswith('category')].values)
        self.col = torch.Tensor(self.df.loc[:,self.df.columns.str.startswith('color')].values)
        self.img = torch.Tensor(self.df.loc[:,self.df.columns.str.startswith('img')].values)
        self.txt = torch.Tensor(self.df.loc[:,self.df.columns.str.startswith('text')].values)

        if opt_lambda == None:
            sales, self.opt_lambda = boxcox(self.df['sales_total'])
        else:
            self.opt_lambda = opt_lambda
            sales = boxcox(self.df['sales_total'], opt_lambda)
        self.y = torch.Tensor(sales)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        fabric = self.fab[idx]
        category = self.cat[idx]
        color = self.col[idx]
        image = self.img[idx]
        text = self.txt[idx]
        sales_total = self.y[idx]

        return fabric, category, color, image, text, sales_total

In [5]:
from model import PositionalEncoding

In [13]:
class StaticFeatureEncoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, use_img, use_text, num_layers, before_meta, dropout=0.2):
        super(StaticFeatureEncoder, self).__init__()

        self.img_linear = nn.Linear(2048, embedding_dim)
        self.text_linear = nn.Linear(768, embedding_dim)
        self.use_img = use_img
        self.use_text = use_text

        self.before_meta = before_meta

        self.linear = nn.Linear(535, 512)

        self.batchnorm = nn.BatchNorm1d(hidden_dim)

        self.feature_fusion = nn.Sequential(
            nn.Linear(1, hidden_dim, bias=False),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.pos_embedding = PositionalEncoding(hidden_dim, max_len=512)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4, dropout=0.2)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)


    # def forward(self, img_encoding, text_encoding):
    def forward(self, image, text, meta_data=None):
        # Fuse static features together
        img_encoding = self.img_linear(image)
        text_encoding = self.text_linear(text)

        # Build input
        decoder_inputs = []
        if self.use_img == 1:
            decoder_inputs.append(img_encoding)
        if self.use_text == 1:
            decoder_inputs.append(text_encoding)
        if self.before_meta:
            decoder_inputs.append(meta_data)

        concat_features = torch.cat(decoder_inputs, dim=1)

        if self.before_meta:
            concat_features = self.linear(concat_features)

        print(concat_features.size())
        features = self.batchnorm(concat_features)
        features = self.feature_fusion(features.unsqueeze(2))
        features = self.pos_embedding(features)
        features = self.encoder(features)

        return features


class Scale_factor_layer_encoder(nn.Module):
    def __init__(self, hidden_dim, before_meta, dropout):
        super().__init__()

        self.dropout = nn.Dropout(p=dropout)

        self.conv1 = nn.Conv2d(1, 1, 3, stride=2, bias=True)
        self.conv2 = nn.Conv2d(1, 1, 3, stride=2, bias=True)
        self.conv3 = nn.Conv2d(1, 1, 3, stride=2, bias=True)
        self.conv4 = nn.Conv2d(1, 1, 3, stride=2, bias=True)
        self.conv5 = nn.Conv2d(1, 1, 3, stride=2, bias=True)
        self.conv6 = nn.Conv2d(1, 1, 3, stride=2, bias=True)


        self.meta_linear1 = nn.Linear(23, 49, bias=True)

        self.before_meta = before_meta

        if before_meta:
            self.linear = nn.Linear(49, 1, bias=True)
        else:
            self.linear = nn.Linear(98, 1, bias=True)

        self.activation = nn.GELU()
        # self.activation = nn.ReLU()

        self.norm1 = nn.LayerNorm(255)
        self.norm2 = nn.LayerNorm(127)
        self.norm3 = nn.LayerNorm(63)
        self.norm4 = nn.LayerNorm(31)
        self.norm5 = nn.LayerNorm(15)
        self.norm6 = nn.LayerNorm(7)

        # self.pool = nn.AdaptiveAvgPool2d((64, 64))

    # def forward(self, encoder_out, meta_data):
    def forward(self, encoder_out, meta_data=None):
        out = self.norm1(self.conv1(encoder_out))
        out = self.norm2(self.conv2(out))
        out = self.activation(out)
        out = self.dropout(out)

        out = self.norm3(self.conv3(out))
        out = self.norm4(self.conv4(out))
        out = self.activation(out)
        out = self.dropout(out)

        out = self.norm5(self.conv5(out))
        out = self.norm6(self.conv6(out))
        out = self.activation(out)
        out = self.dropout(out)


        out = out.squeeze().reshape(out.shape[0], -1)

        if self.before_meta:
           pass
        else:
            out_meta = self.activation(self.meta_linear1(meta_data))
            out = torch.cat([out, out_meta], dim=1)

        out = self.dropout(out)
        out = self.linear(out)


        return out


class OnlyStaticFeature(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, use_img, use_text, num_layers, before_meta):
        super().__init__()
        self.static_feature_encoder = StaticFeatureEncoder(embedding_dim, hidden_dim, use_img, use_text, num_layers, before_meta)
        self.scale_factor_layer_encoder = Scale_factor_layer_encoder(hidden_dim, before_meta, dropout=0.2)
    
    def forward(self, image, text, meta_data):
        static_feature_fusion = self.static_feature_encoder(image, text)
        output = self.scale_factor_layer_encoder(static_feature_fusion.unsqueeze(1), meta_data)
        return output

In [14]:
from torchmetrics.regression import R2Score, SymmetricMeanAbsolutePercentageError

r2score = R2Score()
smape = SymmetricMeanAbsolutePercentageError()
device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = OnlyStaticFeature(
    args.embedding_dim, 
    args.hidden_dim, 
    args.use_img, 
    args.use_text, 
    args.num_hidden_layers, 
    args.before_meta
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

train_dataset = MetaDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False, num_workers=4)

test_dataset = MetaDataset(test_df, train_dataset.opt_lambda)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=4)

/home/smart01/.conda/envs/bonbak/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [15]:
def get_metric(target, prediction, opt_lambda):
    prediction = inv_boxcox(prediction.detach().cpu(), opt_lambda)
    target = inv_boxcox(target.detach().cpu(), opt_lambda)
    
    r2_score = torch.mean(torch.stack(
        [r2score(prediction[i], target[i]) for i in
            range(len(target))]))

    ad_smape = torch.mean(torch.stack(
        [smape(prediction[i], target[i]) * 0.5 for i
            in range(len(target))]))

    return r2_score, ad_smape


def train(model, train_loader, device):
    model.train()
    total_mse = 0
    total_rs = 0
    total_smp = 0

    for train_batch in train_loader:
        fabric, category, color, image, text, sale = train_batch
        meta = torch.cat([fabric, category, color], axis=1)

        image = image.to(device)
        text = text.to(device)
        meta = meta.to(device)
        sale = sale.to(device)

        optimizer.zero_grad()
        pred = model(image, text, meta).squeeze()
        loss = F.mse_loss(sale, pred)

        loss.backward()
        optimizer.step()

        r2_score, ad_smape = get_metric(sale.unsqueeze(0), pred.unsqueeze(0), train_loader.dataset.opt_lambda)
        mse = loss.item()

        total_mse += sale.shape[0] * mse
        total_rs += sale.shape[0] * r2_score
        total_smp += sale.shape[0] * ad_smape
        
    mse = total_mse / len(train_loader.dataset)
    r2_score = total_rs / len(train_loader.dataset)
    ad_smape = total_smp / len(train_loader.dataset)

    print(f'Train\t| r2_score: {r2_score:.4f}\tad_smape: {ad_smape:.4f}\tmse: {mse:.4f}')

def test(model, test_loader, device):
    model.eval()

    total_mse = 0
    total_rs = 0
    total_smp = 0

    with torch.no_grad():
        for test_batch in test_loader:
            fabric, category, color, image, text, sale = test_batch
            meta = torch.cat([fabric, category, color], axis=1)

            image = image.to(device)
            text = text.to(device)
            meta = meta.to(device)
            sale = sale.to(device)

            pred = model(image, text, meta).squeeze()
            loss = F.mse_loss(sale, pred)

            r2_score, ad_smape = get_metric(sale.unsqueeze(0), pred.unsqueeze(0), test_loader.dataset.opt_lambda)
            mse = loss.item()

            total_mse += sale.shape[0] * mse
            total_rs += sale.shape[0] * r2_score
            total_smp += sale.shape[0] * ad_smape

    
    mse = total_mse / len(test_loader.dataset)
    r2_score = total_rs / len(test_loader.dataset)
    ad_smape = total_smp / len(test_loader.dataset)

    print(f'Test\t| r2_score: {r2_score:.4f}\tad_smape: {ad_smape:.4f}\tmse: {mse:.4f}')

In [16]:
for epoch in range(1, args.epochs+1):
    print(f'Epoch {epoch}')
    train(model, train_loader, device)
    test(model, test_loader, device)

Epoch 1
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
to

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])